In [1]:
# import modules
import pandas as pd
import pickle
import gensim
import numpy as np
import string
from opencc import OpenCC
import ckip
import jieba
# Path of files
SENTENCE_DICT = "../../pickle/sentence_dict.pickle"
WORDVEC_MODEL = '../../wordvec_model/'
# Variables
DEMENTIA_NUM = 51
CONTROL_NUM = 51
WV_DIIM = 500

def read_sentence_file(file_name=None):
    with open(SENTENCE_DICT, 'rb') as f:
        sentence_dict = pickle.load(f)
        print("Load sentence text data ...")
    return sentence_dict

def load_wordvec_model(file_name):
    w2v_model = gensim.models.Word2Vec.load(WORDVEC_MODEL+file_name)
    words = []
    for word in w2v_model.wv.vocab:
        words.append(word)
    print('Load word2vec model sucess ...')
    print('Number of token: {}'.format(len(words)))
    print('Dimensions of word vector: {}'.format(len(w2v_model[words[0]])))
    return w2v_model

In [2]:
sentence_dict = read_sentence_file()

Load sentence text data ...


In [3]:
w2v_model = load_wordvec_model('500features_20context_20mincount')

Load word2vec model sucess ...
Number of token: 259638
Dimensions of word vector: 500


/home/yyliu/anaconda3/envs/NLP/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [4]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, LSTM, Embedding, Input

In [5]:
def split_punctuation(sentence):
    punctuation = set(string.punctuation+"，"+"、"+"」"+"「"+"。"+" "+"！")
    sentence_split = []
    tmp = ''
    for i in sentence:
        if i not in punctuation:
            tmp += i
        else:
            sentence_split.append(tmp)
            tmp = ''
    return sentence_split
sentence = '3個人，一個媽媽兩個小孩，小孩站在椅子上要拿西點，椅子都快倒下來了，在拿這個西點餅乾要吃，手下還拿著一塊，'
print(split_punctuation(sentence))

['3個人', '一個媽媽兩個小孩', '小孩站在椅子上要拿西點', '椅子都快倒下來了', '在拿這個西點餅乾要吃', '手下還拿著一塊']


In [7]:
with open('../../data/dementia.txt', encoding='utf8') as f:
    dementia_txt = f.readlines()
sentence = []
for i in range(len(dementia_txt)):
    if i%2==0:
        sentence.extend(split_punctuation(dementia_txt[i+1]))
dementia_num = len(sentence)
with open('../../data/control_51.txt', encoding='utf8') as f:
    control_txt = f.readlines()
for i in range(len(control_txt)):
    if i%2==0:
        sentence.extend(split_punctuation(control_txt[i+1]))
control_num = len(sentence) - dementia_num
############
# train set#
############
train_data = np.array(sentence)
dementia_labels = [[0, 1] for _ in train_data[:dementia_num]]
control_labels = [[1, 0] for _ in train_data[dementia_num:]]
print('total number of train set: {}'.format(train_data.shape[0]))
print('sentence number of dementia subject: {}'.format(len(dementia_labels)))
print('sentence number of control normal subject: {}'.format(len(control_labels)))

total number of train set: 873
sentence number of dementia subject: 442
sentence number of control normal subject: 431


In [8]:
JIEBA_DICT = '../../data/dict.txt.big'
jieba.set_dictionary(JIEBA_DICT)
train_data_seg = []
for i in train_data:
    train_data_seg.append(' '.join(jieba.lcut(i)))
print(train_data_seg[5])

Building prefix dict from /home/yyliu/code/NLP/data/dict.txt.big ...
Loading model from cache /tmp/jieba.u74f96b08eeb68fe4b0ac4c13a6f276ed.cache
Loading model cost 1.243 seconds.
Prefix dict has been built succesfully.


手下 還拿著 一塊


In [9]:
SEQUENCE_LENGTH = 17
VOCAB_DIM = 500
train_data_seg_array = np.array(train_data_seg)
l = []
for i in range(len(train_data_seg_array)):
    l.append(len(train_data_seg_array[i].split(' ')))
#     if len(train_data_seg_array[i])==1:
#         print(i, train_data_seg_array[i])
print('Max token number of sentence: {}'.format(np.max(l)))
print('Min token number of sentence: {}'.format(np.min(l)))
print('Mean token number of sentence: {}'.format(np.mean(l)))

Max token number of sentence: 17
Min token number of sentence: 1
Mean token number of sentence: 5.747995418098511


In [10]:
train_vec = []
for s in train_data_seg:
    token_list = []
    for token in s.split(' '):
        if token in w2v_model.wv.vocab:
            token_list.append(np.asarray(w2v_model.wv[token]))
    if len(token_list) < SEQUENCE_LENGTH:
        for i in range(SEQUENCE_LENGTH - len(token_list)):
            token_list.append(np.zeros(shape=VOCAB_DIM))
    train_vec.append([token_list[0:SEQUENCE_LENGTH]])
# seg_sentence_vec = []
# for key, s in seg_sentence.items():
#     token_list = []
#     for token in s:
#         if token in w2v_model.wv.vocab:
#             token_list.append(np.asarray(w2v_model.wv[token]))
#     if len(token_list) < SEQUENCE_LENGTH:
#         for i in range(SEQUENCE_LENGTH - len(token_list)):
#             token_list.append(np.zeros(shape=VOCAB_DIM))
# #             token_list.append(np.zeros(shape=(VOCAB_DIM, 1)).tolist())
#     seg_sentence_vec.append([token_list[0:SEQUENCE_LENGTH]])

In [11]:
train_vec = np.asarray(train_vec).reshape(len(train_data_seg),-1,500)
print(train_vec.shape)
print(train_vec[2])

(873, 17, 500)
[[-7.92692825e-02  1.57746319e-02  5.27736768e-02 ...  3.76927592e-02
   7.03343600e-02  3.61793354e-04]
 [ 1.03810102e-01 -3.23520899e-02  1.29130320e-03 ... -2.70457361e-02
  -3.82427163e-02  1.65345892e-02]
 [-1.31187662e-02 -1.90271542e-03 -6.02233633e-02 ...  2.55492123e-05
   2.02863179e-02  1.29380105e-02]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [12]:
import tensorflow as tf

In [13]:
input_x = tf.placeholder(tf.int32, [None, SEQUENCE_LENGTH], name='input_x')
input_y = tf.placeholder(tf.float32, [None, 2], name='input_y')
dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
l2_loss = tf.constant(0.0)

In [14]:
max_sentence_length = max([len(x.split(' ')) for x in train_data_seg_array])
print(max_sentence_length)
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_sentence_length)
x_one_hot = np.array(list(vocab_processor.fit_transform(train_data_seg_array)))

17


In [15]:
vocab_size=len(vocab_processor.vocabulary_)
embedding_size = 128
with tf.device('/cpu:0'), tf.name_scope('embedding'):
    W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name = 'W')
    embedded_chars = tf.nn.embedding_lookup(W, input_x)
    embedded_chars_expand = tf.expand_dims(embedded_chars, -1)

In [16]:
import tensorflow as tf
import numpy as np


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, SEQUENCE_LENGTH], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
        

In [18]:
y = np.concatenate([dementia_labels, control_labels], 0)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x_one_hot[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [20]:
dev_sample_index = -1 * int(.1*float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
del x_shuffled, y_shuffled

In [21]:
print('vocab_size: {}'.format(len(vocab_processor.vocabulary_)))
print('Train/Dev split : {}/{}'.format(len(y_train), len(y_dev)))

vocab_size: 961
Train/Dev split : 786/87


In [22]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [29]:
import time
import os
import datetime
batch_size = 32
num_epochs = 200
dropout_keep_prob = 0.5

num_checkpoints = 5
checkpoint_every = 100

evaluate_every = 100

filter_sizes = (3,4,5)
num_filters = 128
l2_reg_lambda = 0.0
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = TextCNN(sequence_length=SEQUENCE_LENGTH, 
                     num_classes=2, 
                     vocab_size=len(vocab_processor.vocabulary_), 
                     embedding_size=embedding_size, 
                     filter_sizes=filter_sizes, 
                     num_filters=num_filters, 
                      l2_reg_lambda=l2_reg_lambda)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))
        sess.run(tf.global_variables_initializer())
        
        def train_step(x_batch, y_batch):
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
        

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

Saved model checkpoint to /home/yyliu/code/NLP/src/jupyter_notebook/runs/1525181871/checkpoints/model-100

2018-05-01T21:37:54.027947: step 101, loss 0.16316, acc 0.9375
2018-05-01T21:37:54.043172: step 102, loss 0.306615, acc 0.90625
2018-05-01T21:37:54.057442: step 103, loss 0.324804, acc 0.8125
2018-05-01T21:37:54.073430: step 104, loss 0.327534, acc 0.84375
2018-05-01T21:37:54.087574: step 105, loss 0.475429, acc 0.875
2018-05-01T21:37:54.100604: step 106, loss 0.396569, acc 0.75
2018-05-01T21:37:54.113357: step 107, loss 0.473461, acc 0.8125
2018-05-01T21:37:54.128713: step 108, loss 0.397362, acc 0.84375
2018-05-01T21:37:54.143106: step 109, loss 0.195337, acc 0.90625
2018-05-01T21:37:54.155853: step 110, loss 0.492219, acc 0.78125
2018-05-01T21:37:54.170866: step 111, loss 0.175041, acc 0.90625
2018-05-01T21:37:54.186774: step 112, loss 0.397323, acc 0.8125
2018-05-01T21:37:54.200371: step 113, loss 0.17666, acc 0.9375
2018-05-01T21:37:54.214107: step 114, loss 0.137764, acc 1
2

2018-05-01T21:37:56.107953: step 238, loss 0.0873921, acc 0.96875
2018-05-01T21:37:56.120854: step 239, loss 0.105385, acc 0.96875
2018-05-01T21:37:56.134801: step 240, loss 0.363075, acc 0.90625
2018-05-01T21:37:56.147830: step 241, loss 0.244517, acc 0.90625
2018-05-01T21:37:56.160614: step 242, loss 0.0317824, acc 1
2018-05-01T21:37:56.173915: step 243, loss 0.100873, acc 0.96875
2018-05-01T21:37:56.186382: step 244, loss 0.206963, acc 0.90625
2018-05-01T21:37:56.205572: step 245, loss 0.177771, acc 0.90625
2018-05-01T21:37:56.221448: step 246, loss 0.0469501, acc 1
2018-05-01T21:37:56.236960: step 247, loss 0.0664782, acc 0.96875
2018-05-01T21:37:56.254445: step 248, loss 0.0877527, acc 0.96875
2018-05-01T21:37:56.270039: step 249, loss 0.0467555, acc 0.96875
2018-05-01T21:37:56.282239: step 250, loss 0.04831, acc 1
2018-05-01T21:37:56.296027: step 251, loss 0.138856, acc 0.96875
2018-05-01T21:37:56.309461: step 252, loss 0.0418017, acc 1
2018-05-01T21:37:56.324148: step 253, loss 

2018-05-01T21:37:58.198237: step 378, loss 0.0466744, acc 1
2018-05-01T21:37:58.212061: step 379, loss 0.282856, acc 0.90625
2018-05-01T21:37:58.225423: step 380, loss 0.0695216, acc 1
2018-05-01T21:37:58.238544: step 381, loss 0.0624709, acc 0.96875
2018-05-01T21:37:58.253150: step 382, loss 0.0223133, acc 1
2018-05-01T21:37:58.266931: step 383, loss 0.0826437, acc 0.96875
2018-05-01T21:37:58.280665: step 384, loss 0.0739082, acc 0.9375
2018-05-01T21:37:58.296588: step 385, loss 0.0347336, acc 1
2018-05-01T21:37:58.309837: step 386, loss 0.0458677, acc 0.96875
2018-05-01T21:37:58.323937: step 387, loss 0.0373501, acc 1
2018-05-01T21:37:58.336314: step 388, loss 0.0462491, acc 1
2018-05-01T21:37:58.348709: step 389, loss 0.0746973, acc 0.96875
2018-05-01T21:37:58.362135: step 390, loss 0.0183565, acc 1
2018-05-01T21:37:58.374884: step 391, loss 0.0638987, acc 0.96875
2018-05-01T21:37:58.389459: step 392, loss 0.0799711, acc 0.96875
2018-05-01T21:37:58.402253: step 393, loss 0.0831333, 

2018-05-01T21:38:00.093613: step 507, loss 0.0373303, acc 1
2018-05-01T21:38:00.109715: step 508, loss 0.0154886, acc 1
2018-05-01T21:38:00.122947: step 509, loss 0.0229952, acc 1
2018-05-01T21:38:00.136118: step 510, loss 0.0673041, acc 0.96875
2018-05-01T21:38:00.149558: step 511, loss 0.0137748, acc 1
2018-05-01T21:38:00.163173: step 512, loss 0.118633, acc 0.96875
2018-05-01T21:38:00.176538: step 513, loss 0.0539771, acc 0.96875
2018-05-01T21:38:00.192585: step 514, loss 0.0160605, acc 1
2018-05-01T21:38:00.205886: step 515, loss 0.0571255, acc 0.96875
2018-05-01T21:38:00.219960: step 516, loss 0.0243812, acc 1
2018-05-01T21:38:00.233651: step 517, loss 0.0701495, acc 0.96875
2018-05-01T21:38:00.247775: step 518, loss 0.051497, acc 1
2018-05-01T21:38:00.265382: step 519, loss 0.163866, acc 0.9375
2018-05-01T21:38:00.278595: step 520, loss 0.00532347, acc 1
2018-05-01T21:38:00.291972: step 521, loss 0.00899686, acc 1
2018-05-01T21:38:00.305886: step 522, loss 0.0685696, acc 0.96875


...

2018-05-01T21:39:07.134009: step 4356, loss 0.000254996, acc 1
2018-05-01T21:39:07.150661: step 4357, loss 0.0290883, acc 1
2018-05-01T21:39:07.167170: step 4358, loss 0.0201149, acc 1
2018-05-01T21:39:07.181180: step 4359, loss 0.0862802, acc 0.96875
2018-05-01T21:39:07.195334: step 4360, loss 0.0028394, acc 1
2018-05-01T21:39:07.209097: step 4361, loss 0.0110038, acc 1
2018-05-01T21:39:07.223043: step 4362, loss 0.00321476, acc 1
2018-05-01T21:39:07.236211: step 4363, loss 0.0139999, acc 1
2018-05-01T21:39:07.250444: step 4364, loss 0.00136825, acc 1
2018-05-01T21:39:07.265454: step 4365, loss 0.00130746, acc 1
2018-05-01T21:39:07.278993: step 4366, loss 0.00121438, acc 1
2018-05-01T21:39:07.293148: step 4367, loss 0.000131492, acc 1
2018-05-01T21:39:07.307136: step 4368, loss 0.0158331, acc 1
2018-05-01T21:39:07.321338: step 4369, loss 0.021585, acc 1
2018-05-01T21:39:07.334871: step 4370, loss 5.92641e-05, acc 1
2018-05-01T21:39:07.350841: step 4371, loss 0.000509244, acc 1
2018-05

2018-05-01T21:39:09.135369: step 4488, loss 0.00157054, acc 1
2018-05-01T21:39:09.152410: step 4489, loss 0.000164917, acc 1
2018-05-01T21:39:09.167179: step 4490, loss 0.0412875, acc 0.96875
2018-05-01T21:39:09.181877: step 4491, loss 0.00703151, acc 1
2018-05-01T21:39:09.195651: step 4492, loss 0.109532, acc 0.9375
2018-05-01T21:39:09.209516: step 4493, loss 4.47503e-05, acc 1
2018-05-01T21:39:09.224091: step 4494, loss 0.000443768, acc 1
2018-05-01T21:39:09.238463: step 4495, loss 0.000451917, acc 1
2018-05-01T21:39:09.251407: step 4496, loss 7.50515e-05, acc 1
2018-05-01T21:39:09.264801: step 4497, loss 0.00309127, acc 1
2018-05-01T21:39:09.278117: step 4498, loss 0.0083569, acc 1
2018-05-01T21:39:09.291872: step 4499, loss 8.29487e-05, acc 1
2018-05-01T21:39:09.304394: step 4500, loss 0.000388715, acc 1

Evaluation:
2018-05-01T21:39:09.308790: step 4500, loss 0.730549, acc 0.804598

Saved model checkpoint to /home/yyliu/code/NLP/src/jupyter_notebook/runs/1525181871/checkpoints/mod

2018-05-01T21:39:11.247517: step 4624, loss 0.000484658, acc 1
2018-05-01T21:39:11.263324: step 4625, loss 0.000488224, acc 1
2018-05-01T21:39:11.282432: step 4626, loss 0.000345991, acc 1
2018-05-01T21:39:11.296191: step 4627, loss 0.0753264, acc 0.96875
2018-05-01T21:39:11.310110: step 4628, loss 0.005478, acc 1
2018-05-01T21:39:11.324110: step 4629, loss 0.00667853, acc 1
2018-05-01T21:39:11.337432: step 4630, loss 0.0918773, acc 0.96875
2018-05-01T21:39:11.351386: step 4631, loss 0.000440997, acc 1
2018-05-01T21:39:11.365258: step 4632, loss 0.00174644, acc 1
2018-05-01T21:39:11.379624: step 4633, loss 0.0270288, acc 0.96875
2018-05-01T21:39:11.393962: step 4634, loss 0.00149682, acc 1
2018-05-01T21:39:11.408190: step 4635, loss 0.00162981, acc 1
2018-05-01T21:39:11.425637: step 4636, loss 0.123764, acc 0.96875
2018-05-01T21:39:11.440735: step 4637, loss 0.00133346, acc 1
2018-05-01T21:39:11.457688: step 4638, loss 0.015904, acc 1
2018-05-01T21:39:11.473877: step 4639, loss 0.00022

2018-05-01T21:39:13.344948: step 4763, loss 0.0240451, acc 0.96875
2018-05-01T21:39:13.362311: step 4764, loss 0.0150711, acc 1
2018-05-01T21:39:13.379750: step 4765, loss 0.0146201, acc 1
2018-05-01T21:39:13.393852: step 4766, loss 0.00554908, acc 1
2018-05-01T21:39:13.407845: step 4767, loss 0.0436817, acc 0.96875
2018-05-01T21:39:13.421186: step 4768, loss 0.000578404, acc 1
2018-05-01T21:39:13.434736: step 4769, loss 3.65912e-05, acc 1
2018-05-01T21:39:13.448787: step 4770, loss 0.000657839, acc 1
2018-05-01T21:39:13.464190: step 4771, loss 6.76368e-05, acc 1
2018-05-01T21:39:13.477484: step 4772, loss 0.00364073, acc 1
2018-05-01T21:39:13.490971: step 4773, loss 0.00846415, acc 1
2018-05-01T21:39:13.504831: step 4774, loss 0.00558604, acc 1
2018-05-01T21:39:13.517869: step 4775, loss 8.2163e-05, acc 1
2018-05-01T21:39:13.530778: step 4776, loss 8.56865e-05, acc 1
2018-05-01T21:39:13.544575: step 4777, loss 0.000572235, acc 1
2018-05-01T21:39:13.568198: step 4778, loss 0.000473295,

2018-05-01T21:39:15.282659: step 4892, loss 0.000231587, acc 1
2018-05-01T21:39:15.305233: step 4893, loss 0.00580374, acc 1
2018-05-01T21:39:15.320147: step 4894, loss 0.00185575, acc 1
2018-05-01T21:39:15.333928: step 4895, loss 0.000106495, acc 1
2018-05-01T21:39:15.347451: step 4896, loss 0.000195675, acc 1
2018-05-01T21:39:15.360893: step 4897, loss 0.000656128, acc 1
2018-05-01T21:39:15.374492: step 4898, loss 0.0544328, acc 0.96875
2018-05-01T21:39:15.388235: step 4899, loss 0.00191773, acc 1
2018-05-01T21:39:15.401245: step 4900, loss 6.221e-05, acc 1

Evaluation:
2018-05-01T21:39:15.405536: step 4900, loss 0.763266, acc 0.83908

Saved model checkpoint to /home/yyliu/code/NLP/src/jupyter_notebook/runs/1525181871/checkpoints/model-4900

2018-05-01T21:39:15.469513: step 4901, loss 0.00019933, acc 1
2018-05-01T21:39:15.482965: step 4902, loss 0.00734048, acc 1
2018-05-01T21:39:15.502183: step 4903, loss 0.131668, acc 0.96875
2018-05-01T21:39:15.517950: step 4904, loss 0.000180298,